# Project 4: Team 7
## Predicting Congressional Bill Passage
### Extract,Transform, and Load: Congress data

#### Import dependecies and read in data:

In [1]:
# Import Dependencies:
import pandas as pd
import numpy as np

In [7]:
# Read in master data from S3 bucket on AWS:
master_df = pd.read_csv('https://project-4-team7.s3.ca-central-1.amazonaws.com/c113_c118_master.csv', index_col=[0])
# # Split data into raw df for the House and Senata data
df_house = master_df[master_df['Legislation Number'].str.contains("H.J|H.R.")==True]
df_senate = master_df[master_df['Legislation Number'].str.contains("S.J|S.")==True]
df_senate.reset_index(drop=True)

# Save raw house and senate files to .csv:
# df_house.to_csv('../Resources/House_113_118.csv', index=False)
# df_senate.to_csv('../Resources/Senate_113_118.csv', index=False)
# Load all of these files to s3 bucket on AWS since they are too big for repo:
# df_house = pd.read_csv('https://project-4-team7.s3.ca-central-1.amazonaws.com/House_113_118.csv', index_col=[0])
# df_senate = pd.read_csv('https://project-4-team7.s3.ca-central-1.amazonaws.com/Senate_113_118.csv', index_col=[0])

# Check master df:
master_df.head()

,Legislation Number,URL,Congress,Title,Amends Bill,Sponsor,Date Offered,Date of Introduction,Number of Cosponsors,Date Submitted,...,Subject.453,Subject.454,Subject.455,Subject.456,Cosponsor.380,Cosponsor.381,Cosponsor.382,Cosponsor.383,Cosponsor.384,Cosponsor.385
0,H.R. 11,https://www.congress.gov/bill/113th-congress/h...,113th Congress (2013-2014),Violence Against Women Reauthorization Act of ...,NaN,"Moore, Gwen [Rep.-D-WI-4]",NaN,01/22/2013,200,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,H.R. 12,https://www.congress.gov/bill/113th-congress/h...,113th Congress (2013-2014),Voter Empowerment Act of 2013,NaN,"Lewis, John [Rep.-D-GA-5]",NaN,01/23/2013,179,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,H.R. 13,https://www.congress.gov/bill/113th-congress/h...,113th Congress (2013-2014),Reserved for the Minority Leader.,NaN,"Pelosi, Nancy [Rep.-D-CA-8]",NaN,01/03/2013,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,H.R. 14,https://www.congress.gov/bill/113th-congress/h...,113th Congress (2013-2014),Reserved for the Minority Leader.,NaN,"Pelosi, Nancy [Rep.-D-CA-8]",NaN,01/03/2013,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,H.R. 15,https://www.congress.gov/bill/113th-congress/h...,113th Congress (2013-2014),"Border Security, Economic Opportunity, and Imm...",NaN,"Garcia, Joe [Rep.-D-FL-26]",NaN,10/02/2013,200,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86280,S.J.Res. 5,https://www.congress.gov/bill/118th-congress/s...,118th Congress (2023-2024),A joint resolution disapproving the action of ...,NaN,"Cotton, Tom [Sen.-R-AR]",NaN,01/31/2023,27,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86281,S.J.Res. 4,https://www.congress.gov/bill/118th-congress/s...,118th Congress (2023-2024),A joint resolution removing the deadline for t...,NaN,"Cardin, Benjamin L. [Sen.-D-MD]",NaN,01/24/2023,52,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86282,S.J.Res. 3,https://www.congress.gov/bill/118th-congress/s...,118th Congress (2023-2024),A joint resolution proposing an amendment to t...,NaN,"Tester, Jon [Sen.-D-MT]",NaN,01/23/2023,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86283,S.J.Res. 2,https://www.congress.gov/bill/118th-congress/s...,118th Congress (2023-2024),A joint resolution proposing an amendment to t...,NaN,"Cruz, Ted [Sen.-R-TX]",NaN,01/23/2023,14,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
#Get list of cosponsor columns
cosponsor_cols = [col for col in master_df.columns if 'Cosponsor' in col]
cosponsor_cols.remove('Number of Cosponsors')
print(cosponsor_cols)

['Cosponsor', 'Cosponsor.1', 'Cosponsor.2', 'Cosponsor.3', 'Cosponsor.4', 'Cosponsor.5', 'Cosponsor.6', 'Cosponsor.7', 'Cosponsor.8', 'Cosponsor.9', 'Cosponsor.10', 'Cosponsor.11', 'Cosponsor.12', 'Cosponsor.13', 'Cosponsor.14', 'Cosponsor.15', 'Cosponsor.16', 'Cosponsor.17', 'Cosponsor.18', 'Cosponsor.19', 'Cosponsor.20', 'Cosponsor.21', 'Cosponsor.22', 'Cosponsor.23', 'Cosponsor.24', 'Cosponsor.25', 'Cosponsor.26', 'Cosponsor.27', 'Cosponsor.28', 'Cosponsor.29', 'Cosponsor.30', 'Cosponsor.31', 'Cosponsor.32', 'Cosponsor.33', 'Cosponsor.34', 'Cosponsor.35', 'Cosponsor.36', 'Cosponsor.37', 'Cosponsor.38', 'Cosponsor.39', 'Cosponsor.40', 'Cosponsor.41', 'Cosponsor.42', 'Cosponsor.43', 'Cosponsor.44', 'Cosponsor.45', 'Cosponsor.46', 'Cosponsor.47', 'Cosponsor.48', 'Cosponsor.49', 'Cosponsor.50', 'Cosponsor.51', 'Cosponsor.52', 'Cosponsor.53', 'Cosponsor.54', 'Cosponsor.55', 'Cosponsor.56', 'Cosponsor.57', 'Cosponsor.58', 'Cosponsor.59', 'Cosponsor.60', 'Cosponsor.61', 'Cosponsor.62', 'Co

In [48]:
# Create new df with cosponsor columns
cosponsors_df = master_df[cosponsor_cols]
cosponsors_df.head()

,Cosponsor,Cosponsor.1,Cosponsor.2,Cosponsor.3,Cosponsor.4,Cosponsor.5,Cosponsor.6,Cosponsor.7,Cosponsor.8,Cosponsor.9,...,Cosponsor.376,Cosponsor.377,Cosponsor.378,Cosponsor.379,Cosponsor.380,Cosponsor.381,Cosponsor.382,Cosponsor.383,Cosponsor.384,Cosponsor.385
0,"Sewell, Terri A. [Rep.-D-AL-7]","Faleomavaega, Eni F. H. [Del.-D-AS-At Large]","Kirkpatrick, Ann [Rep.-D-AZ-1]","Grijalva, Raúl M. [Rep.-D-AZ-3]","Sinema, Kyrsten [Rep.-D-AZ-9]","Huffman, Jared [Rep.-D-CA-2]","Matsui, Doris O. [Rep.-D-CA-6]","Bera, Ami [Rep.-D-CA-7]","McNerney, Jerry [Rep.-D-CA-9]","Miller, George [Rep.-D-CA-11]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Velazquez, Nydia M. [Rep.-D-NY-7]","Jeffries, Hakeem S. [Rep.-D-NY-8]","Clarke, Yvette D. [Rep.-D-NY-9]","Nadler, Jerrold [Rep.-D-NY-10]","Maloney, Carolyn B. [Rep.-D-NY-12]","Rangel, Charles B. [Rep.-D-NY-13]","Crowley, Joseph [Rep.-D-NY-14]","Serrano, Jose E. [Rep.-D-NY-15]","Lowey, Nita M. [Rep.-D-NY-17]","Tonko, Paul [Rep.-D-NY-20]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Kirkpatrick, Ann [Rep.-D-AZ-1]","Barber, Ron [Rep.-D-AZ-2]","Pastor, Ed [Rep.-D-AZ-7]","Sinema, Kyrsten [Rep.-D-AZ-9]","Huffman, Jared [Rep.-D-CA-2]","Garamendi, John [Rep.-D-CA-3]","Thompson, Mike [Rep.-D-CA-5]","Matsui, Doris O. [Rep.-D-CA-6]","Pelosi, Nancy [Rep.-D-CA-12]","Speier, Jackie [Rep.-D-CA-14]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
# Add bill and congress for identification, number of cosponsors to ensure party counts total correct
cosponsors_df.insert(0, "Legislation Number", master_df['Legislation Number'])
cosponsors_df.insert(1, "Congress", master_df['Congress'])
cosponsors_df.insert(2, "Number of Cosponsors", master_df['Number of Cosponsors'])
cosponsors_df.head()

,Legislation Number,Congress,Number of Cosponsors,Cosponsor,Cosponsor.1,Cosponsor.2,Cosponsor.3,Cosponsor.4,Cosponsor.5,Cosponsor.6,...,Cosponsor.376,Cosponsor.377,Cosponsor.378,Cosponsor.379,Cosponsor.380,Cosponsor.381,Cosponsor.382,Cosponsor.383,Cosponsor.384,Cosponsor.385
0,H.R. 11,113th Congress (2013-2014),200,"Sewell, Terri A. [Rep.-D-AL-7]","Faleomavaega, Eni F. H. [Del.-D-AS-At Large]","Kirkpatrick, Ann [Rep.-D-AZ-1]","Grijalva, Raúl M. [Rep.-D-AZ-3]","Sinema, Kyrsten [Rep.-D-AZ-9]","Huffman, Jared [Rep.-D-CA-2]","Matsui, Doris O. [Rep.-D-CA-6]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,H.R. 12,113th Congress (2013-2014),179,"Velazquez, Nydia M. [Rep.-D-NY-7]","Jeffries, Hakeem S. [Rep.-D-NY-8]","Clarke, Yvette D. [Rep.-D-NY-9]","Nadler, Jerrold [Rep.-D-NY-10]","Maloney, Carolyn B. [Rep.-D-NY-12]","Rangel, Charles B. [Rep.-D-NY-13]","Crowley, Joseph [Rep.-D-NY-14]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,H.R. 13,113th Congress (2013-2014),0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,H.R. 14,113th Congress (2013-2014),0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,H.R. 15,113th Congress (2013-2014),200,"Kirkpatrick, Ann [Rep.-D-AZ-1]","Barber, Ron [Rep.-D-AZ-2]","Pastor, Ed [Rep.-D-AZ-7]","Sinema, Kyrsten [Rep.-D-AZ-9]","Huffman, Jared [Rep.-D-CA-2]","Garamendi, John [Rep.-D-CA-3]","Thompson, Mike [Rep.-D-CA-5]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
# Count cosponsor dems per row and add to df
cosponsor_dems = cosponsors_df.astype(str).apply(lambda x: x.str.contains('-D-')).sum(axis=1)
cosponsors_df['Cosponsor Dems'] = cosponsor_dems


/Users/zackwcrowley/opt/anaconda3/envs/PythonDataOne/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [51]:
# Count cosponsor Reps per row and add to df
cosponsor_reps = cosponsors_df.astype(str).apply(lambda x: x.str.contains('-R-')).sum(axis=1)
cosponsors_df['Cosponsor Reps'] = cosponsor_reps

/Users/zackwcrowley/opt/anaconda3/envs/PythonDataOne/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [52]:
# Count cosponsor Independent per row and add to df
cosponsor_ind = cosponsors_df.astype(str).apply(lambda x: x.str.contains('-I-')).sum(axis=1)
cosponsors_df['Cosponsor Ind'] = cosponsor_ind

/Users/zackwcrowley/opt/anaconda3/envs/PythonDataOne/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [53]:
# get state for each cosponsor
for col in cosponsor_cols:
    cosponsors_df[col].update(cosponsors_df[col].str.split('-').str[2])
cosponsors_df.head()

/Users/zackwcrowley/opt/anaconda3/envs/PythonDataOne/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


,Legislation Number,Congress,Number of Cosponsors,Cosponsor,Cosponsor.1,Cosponsor.2,Cosponsor.3,Cosponsor.4,Cosponsor.5,Cosponsor.6,...,Cosponsor.379,Cosponsor.380,Cosponsor.381,Cosponsor.382,Cosponsor.383,Cosponsor.384,Cosponsor.385,Cosponsor Dems,Cosponsor Reps,Cosponsor Ind
0,H.R. 11,113th Congress (2013-2014),200,AL,AS,AZ,AZ,AZ,CA,CA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,0,0
1,H.R. 12,113th Congress (2013-2014),179,NY,NY,NY,NY,NY,NY,NY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,179,0,0
2,H.R. 13,113th Congress (2013-2014),0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
3,H.R. 14,113th Congress (2013-2014),0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
4,H.R. 15,113th Congress (2013-2014),200,AZ,AZ,AZ,AZ,CA,CA,CA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,197,3,0


In [54]:
# remove any remaining brackets
cosponsors_df[cosponsor_cols] = cosponsors_df[cosponsor_cols].replace({']':''}, regex=True)


/Users/zackwcrowley/opt/anaconda3/envs/PythonDataOne/lib/python3.7/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [55]:
#get count of unique states
cosponsor_states = cosponsors_df[cosponsor_cols].nunique(axis=1)
cosponsors_df['Cosponsor States'] = cosponsor_states


/Users/zackwcrowley/opt/anaconda3/envs/PythonDataOne/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [56]:
# Inspect:
cosponsors_df.head()

,Legislation Number,Congress,Number of Cosponsors,Cosponsor,Cosponsor.1,Cosponsor.2,Cosponsor.3,Cosponsor.4,Cosponsor.5,Cosponsor.6,...,Cosponsor.380,Cosponsor.381,Cosponsor.382,Cosponsor.383,Cosponsor.384,Cosponsor.385,Cosponsor Dems,Cosponsor Reps,Cosponsor Ind,Cosponsor States
0,H.R. 11,113th Congress (2013-2014),200,AL,AS,AZ,AZ,AZ,CA,CA,...,NaN,NaN,NaN,NaN,NaN,NaN,200,0,0,46
1,H.R. 12,113th Congress (2013-2014),179,NY,NY,NY,NY,NY,NY,NY,...,NaN,NaN,NaN,NaN,NaN,NaN,179,0,0,42
2,H.R. 13,113th Congress (2013-2014),0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
3,H.R. 14,113th Congress (2013-2014),0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
4,H.R. 15,113th Congress (2013-2014),200,AZ,AZ,AZ,AZ,CA,CA,CA,...,NaN,NaN,NaN,NaN,NaN,NaN,197,3,0,46


In [57]:
# Create clean df with cosponsor counts
clean_cosponsor_df = cosponsors_df[['Legislation Number','Congress','Number of Cosponsors','Cosponsor Dems','Cosponsor Reps','Cosponsor Ind', 'Cosponsor States']].reset_index(drop=True)


In [58]:
# Join clean cosponsor df with house df
house_df = pd.merge(clean_cosponsor_df, df_house, how='inner', on=['Legislation Number', 'Congress'])
house_df = house_df.drop(columns='Number of Cosponsors_y').rename(columns={'Number of Cosponsors_x': 'Number of Cosponsors'})


In [59]:
# Join clean cosponsor df with senate df
senate_df = pd.merge(clean_cosponsor_df, df_senate, how='inner', on=['Legislation Number', 'Congress'])
senate_df = senate_df.drop(columns='Number of Cosponsors_y').rename(columns={'Number of Cosponsors_x': 'Number of Cosponsors'})


In [60]:
# Concat house and senate dfs to finish cleaning
frames = [house_df, senate_df]
congress_df = pd.concat(frames).reset_index(drop=True)
congress_df.head()

,Legislation Number,Congress,Number of Cosponsors,Cosponsor Dems,Cosponsor Reps,Cosponsor Ind,Cosponsor States,Title,Sponsor,Date of Introduction,Committees,Latest Action,Latest Action Date,Latest Summary,Subject
0,H.R. 11,113th Congress (2013-2014),200,200,0,0,46,Violence Against Women Reauthorization Act of ...,"Moore, Gwen [Rep.-D-WI-4]",01/22/2013,"House - Judiciary, Energy and Commerce, Educat...",Referred to the Subcommittee on Higher Educati...,04/23/2013,<p>Violence Against Women Reauthorization Act...,Accounting and auditing
1,H.R. 12,113th Congress (2013-2014),179,179,0,0,42,Voter Empowerment Act of 2013,"Lewis, John [Rep.-D-GA-5]",01/23/2013,"House - House Administration, Judiciary, Scien...",Referred to the Subcommittee on Higher Educati...,04/23/2013,<p>Voter Empowerment Act of 2013 - Voter Regi...,Administrative law and regulatory procedures
2,H.R. 13,113th Congress (2013-2014),0,0,0,0,0,Reserved for the Minority Leader.,"Pelosi, Nancy [Rep.-D-CA-8]",01/03/2013,NaN,NaN,NaN,NaN,NaN
3,H.R. 14,113th Congress (2013-2014),0,0,0,0,0,Reserved for the Minority Leader.,"Pelosi, Nancy [Rep.-D-CA-8]",01/03/2013,NaN,NaN,NaN,NaN,NaN
4,H.R. 15,113th Congress (2013-2014),200,197,3,0,46,"Border Security, Economic Opportunity, and Imm...","Garcia, Joe [Rep.-D-FL-26]",10/02/2013,"House - Judiciary, Foreign Affairs, Homeland S...",Motion to Discharge Committee filed by Mr. Gar...,03/26/2014,"<p>Border Security, Economic Opportunity, and...",Administrative law and regulatory procedures


In [61]:
# Strip numbers and change Legislation Number to Bill Type
congress_df['Legislation Number'] = congress_df['Legislation Number'].str.replace('\d+', '')
congress_df = congress_df.rename(columns = {"Legislation Number": "Bill Type"})


In [62]:
# Get number of congress only, column 2
congress_df['Congress'] = congress_df['Congress'].str[:3]
# Cast as int64:
congress_df['Congress'] = congress_df['Congress'].astype(int)

In [63]:
# Drop all observations from the 118 Congress, this will be our prediciton dataset:
congress_df = congress_df.drop(congress_df[congress_df['Congress'] == 118].index)
print(congress_df['Congress'].unique())
congress_df

[113 114 115 116 117]


,Bill Type,Congress,Number of Cosponsors,Cosponsor Dems,Cosponsor Reps,Cosponsor Ind,Cosponsor States,Title,Sponsor,Date of Introduction,Committees,Latest Action,Latest Action Date,Latest Summary,Subject
0,H.R.,113,200,200,0,0,46,Violence Against Women Reauthorization Act of ...,"Moore, Gwen [Rep.-D-WI-4]",01/22/2013,"House - Judiciary, Energy and Commerce, Educat...",Referred to the Subcommittee on Higher Educati...,04/23/2013,<p>Violence Against Women Reauthorization Act...,Accounting and auditing
1,H.R.,113,179,179,0,0,42,Voter Empowerment Act of 2013,"Lewis, John [Rep.-D-GA-5]",01/23/2013,"House - House Administration, Judiciary, Scien...",Referred to the Subcommittee on Higher Educati...,04/23/2013,<p>Voter Empowerment Act of 2013 - Voter Regi...,Administrative law and regulatory procedures
2,H.R.,113,0,0,0,0,0,Reserved for the Minority Leader.,"Pelosi, Nancy [Rep.-D-CA-8]",01/03/2013,NaN,NaN,NaN,NaN,NaN
3,H.R.,113,0,0,0,0,0,Reserved for the Minority Leader.,"Pelosi, Nancy [Rep.-D-CA-8]",01/03/2013,NaN,NaN,NaN,NaN,NaN
4,H.R.,113,200,197,3,0,46,"Border Security, Economic Opportunity, and Imm...","Garcia, Joe [Rep.-D-FL-26]",10/02/2013,"House - Judiciary, Foreign Affairs, Homeland S...",Motion to Discharge Committee filed by Mr. Gar...,03/26/2014,"<p>Border Security, Economic Opportunity, and...",Administrative law and regulatory procedures
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55458,S.,117,0,0,0,0,0,A bill to prohibit a COVID-19 vaccination requ...,"Scott, Rick [Sen.-R-FL]",10/19/2021,"Senate - Agriculture, Nutrition, and Forestry",Read twice and referred to the Committee on Ag...,10/19/2021,<p>This bill prohibits the Department of Agri...,Agriculture and Food
55459,S.,117,1,0,1,0,1,Alaska Offshore Parity Act,"Murkowski, Lisa [Sen.-R-AK]",10/19/2021,Senate - Energy and Natural Resources,Placed on Senate Legislative Calendar under Ge...,11/17/2022,NaN,Energy
55460,S.,117,5,0,5,0,5,DEMOCRACIA Act,"Scott, Rick [Sen.-R-FL]",10/18/2021,Senate - Foreign Relations,Read twice and referred to the Committee on Fo...,10/18/2021,<p><strong>Denying Earnings to the Military O...,International Affairs
55461,S.,117,0,0,0,0,0,Parental Consent for Vaccination Act,"Lee, Mike [Sen.-R-UT]",10/18/2021,"Senate - Health, Education, Labor, and Pensions",Read twice and referred to the Committee on He...,10/18/2021,<p><strong>Parental Consent for Vaccination A...,Health


In [64]:
# extract party and state into new column for sponsor
new = congress_df["Sponsor"].str.split("[", n = 1, expand = True)
congress_df['Sponsor Split']= new[1]
congress_df.drop(columns =["Sponsor"], inplace = True)


In [65]:
# Sponsor title, sponsor state, sponsor party in new columns
new2 = congress_df["Sponsor Split"].str.split("-", n = 3, expand = True)
congress_df['Sponsor Title']= new2[0]
congress_df['Sponsor Party']= new2[1]
congress_df['Sponsor State']= new2[2]
congress_df = congress_df.drop(columns={'Sponsor Split'})


In [66]:
# Change date of introduction to month introduced
congress_df.dtypes

Bill Type               object
Congress                 int64
Number of Cosponsors     int64
Cosponsor Dems           int64
Cosponsor Reps           int64
Cosponsor Ind            int64
Cosponsor States         int64
Title                   object
Date of Introduction    object
Committees              object
Latest Action           object
Latest Action Date      object
Latest Summary          object
Subject                 object
Sponsor Title           object
Sponsor Party           object
Sponsor State           object
dtype: object

In [67]:
# Create the month of bill introduction:
congress_df['Date of Introduction'] = pd.to_datetime(congress_df['Date of Introduction'])
congress_df['Month Introduced'] = pd.DatetimeIndex(congress_df['Date of Introduction']).month
congress_df = congress_df.drop(columns={'Date of Introduction'})


In [68]:
# Drop unneeded columns:
congress_df = congress_df.drop(columns={'Title', 'Latest Action Date', 'Latest Summary'})


In [69]:
# Take out extra brackets in statecolumn:
congress_df['Sponsor State'] = congress_df['Sponsor State'].replace({']':''}, regex=True)



In [70]:
congress_df['Sponsor Title'].value_counts()

Rep.                     38314
Sen.                     14780
Del.                       625
Resident Commissioner      132
Name: Sponsor Title, dtype: int64

In [71]:
congress_df['Latest Action'].value_counts()

Referred to the Subcommittee on Health.                                                                                                3728
Referred to the House Committee on Ways and Means.                                                                                     3064
Read twice and referred to the Committee on Finance.                                                                                   2349
Read twice and referred to the Committee on Health, Education, Labor, and Pensions.                                                    1576
Referred to the House Committee on Financial Services.                                                                                 1545
                                                                                                                                       ... 
Read the second time. Placed on Senate Legislative Calendar under General Orders. Calendar No. 190.                                       1
Cloture on the motio

In [75]:
congress_df.shape

(53851, 14)

# Committees Lists for cleaning data:
## House Committees:
#### Standing Committees:
"Agriculture"
"Appropriations"
"Armed Services"
"Budget"
"Education and the Workforce"
"Energy and Commerce"
"Ethics"
"Financial Services"
"Foreign Affairs"
"Homeland Security"
"House Administration"
"Judiciary"
"Natural Resources"
"Oversight and Accountability"
"Rules"
"Science, Space, and Technology"
"Small Business"
"Transportation and Infrastructure"
"Veterans' Affairs"
"Ways and Means"
#### Special, Select, and Other Committees:
"Intelligence"
#### Joint Committees:
"Printing"
"Taxation"
"Library"
"Economic"

## Senate Committees:
#### Standing Committees:
"Agriculture, Nutrition, and Forestry"
"Appropriations"
"Armed Services"
"Banking, Housing, and Urban Affairs"
"Budget"
"Commerce, Science, and Transportation"
"Energy and Natural Resources"
"Environment and Public Works"
"Finance"
"Foreign Relations"
"Health, Education, Labor, and Pensions"
"Homeland Security and Governmental Affairs"
"Judiciary"
"Rules and Administration"
"Small Business and Entrepreneurship"
"Veterans' Affairs"
#### Special, Select, and Other Committees:
"Caucus on International Narcotics Control"
"Ethics" 
"Indian Affairs"
"Intelligence" 

#### Joint Committees:
"Printing"
"Taxation"
"Library"
"Economic"



## Save whole congress dataset up to this point:

In [76]:
# Save whole cleaned dataset:
# congress_df.to_csv('../Resources/cleaned_congress.csv')

### Split the data by House and Senate:

In [77]:
# Split into house and senate dfs:
house_cleaned = congress_df[congress_df['Bill Type'].str.contains("H.J|H")==True]
senate_cleaned = congress_df[congress_df['Bill Type'].str.contains("S.J|S.")==True]
senate_cleaned = senate_cleaned.reset_index(drop=True)
house_cleaned.head()

,Bill Type,Congress,Number of Cosponsors,Cosponsor Dems,Cosponsor Reps,Cosponsor Ind,Cosponsor States,Committees,Latest Action,Subject,Sponsor Title,Sponsor Party,Sponsor State,Month Introduced
0,H.R.,113,200,200,0,0,46,"House - Judiciary, Energy and Commerce, Educat...",Referred to the Subcommittee on Higher Educati...,Accounting and auditing,Rep.,D,WI,1
1,H.R.,113,179,179,0,0,42,"House - House Administration, Judiciary, Scien...",Referred to the Subcommittee on Higher Educati...,Administrative law and regulatory procedures,Rep.,D,GA,1
2,H.R.,113,0,0,0,0,0,NaN,NaN,NaN,Rep.,D,CA,1
3,H.R.,113,0,0,0,0,0,NaN,NaN,NaN,Rep.,D,CA,1
4,H.R.,113,200,197,3,0,46,"House - Judiciary, Foreign Affairs, Homeland S...",Motion to Discharge Committee filed by Mr. Gar...,Administrative law and regulatory procedures,Rep.,D,FL,10


### House data cleaning:

In [78]:
# Committee column recoding to indicator variables:
# Create a list of committees for the House:
house_committees_lst = ["Agriculture", "Appropriations", "Armed Services", "Budget", "Education and the Workforce", "Energy and Commerce", "Ethics", "Financial Services", 
                        "Foreign Affairs", "Homeland Security", "House Administration", "Judiciary", "Natural Resources", 
                        "Oversight and Accountability", "Rules", "Science, Space, and Technology", "Small Business", "Transportation and Infrastructure", 
                        "Veterans' Affairs", "Ways and Means", "Intelligence", "Printing", "Taxation", "Library", "Economic"]
# Run a for loop to set each committee name to a new column and make a dummy var (case=False makes the str.contains case insensitive)):
for comm in house_committees_lst:
    house_cleaned[comm] = np.where(house_cleaned['Committees'].str.contains(comm, case=False), 1, 0)

/Users/zackwcrowley/opt/anaconda3/envs/PythonDataOne/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [38]:
# house_cleaned.head()
# house_cleaned.dtypes
# house_cleaned.apply(pd.value_counts)
# house_cleaned['Committees'].unique()

,Bill Type,Congress,Number of Cosponsors,Cosponsor Dems,Cosponsor Reps,Cosponsor Ind,Cosponsor States,Committees,Latest Action,Subject,...,"Science, Space, and Technology",Small Business,Transportation and Infrastructure,Veterans' Affairs,Ways and Means,Intelligence,Printing,Taxation,Library,Economic
0,H.R.,113,200,200,0,0,46,"House - Judiciary, Energy and Commerce, Educat...",Referred to the Subcommittee on Higher Educati...,Accounting and auditing,...,0,0,0,0,0,0,0,0,0,0
1,H.R.,113,179,179,0,0,42,"House - House Administration, Judiciary, Scien...",Referred to the Subcommittee on Higher Educati...,Administrative law and regulatory procedures,...,1,0,0,1,0,0,0,0,0,0
2,H.R.,113,0,0,0,0,0,NaN,NaN,NaN,...,1,1,1,1,1,1,1,1,1,1
3,H.R.,113,0,0,0,0,0,NaN,NaN,NaN,...,1,1,1,1,1,1,1,1,1,1
4,H.R.,113,200,197,3,0,46,"House - Judiciary, Foreign Affairs, Homeland S...",Motion to Discharge Committee filed by Mr. Gar...,Administrative law and regulatory procedures,...,1,0,1,0,1,0,0,0,0,0


In [79]:
house_cleaned.dtypes

Bill Type                            object
Congress                              int64
Number of Cosponsors                  int64
Cosponsor Dems                        int64
Cosponsor Reps                        int64
Cosponsor Ind                         int64
Cosponsor States                      int64
Committees                           object
Latest Action                        object
Subject                              object
Sponsor Title                        object
Sponsor Party                        object
Sponsor State                        object
Month Introduced                      int64
Agriculture                           int64
Appropriations                        int64
Armed Services                        int64
Budget                                int64
Education and the Workforce           int64
Energy and Commerce                   int64
Ethics                                int64
Financial Services                    int64
Foreign Affairs                 

In [81]:
# Latest Action coding:
# Get "Became Public Law" and name it "bill_passed" set as dummy var:
house_cleaned["bill_passed"] = np.where(house_cleaned['Latest Action'].str.contains("Became Public Law", case=False), 1, 0)
print(house_cleaned["bill_passed"].value_counts())
print(f'For the 113th to 118th House passed {(round((1179/37892)*100,2))}% of laws introduced in the House.')

0    37892
1     1179
Name: bill_passed, dtype: int64
For the 113th to 118th House passed 3.11% of laws introduced in the House.


/Users/zackwcrowley/opt/anaconda3/envs/PythonDataOne/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [83]:
# Latest Action coding:
# Get "referred to the committee" and name it "bill_referred_committee" set as dummy var:
house_cleaned["bill_referred_committee"] = np.where(house_cleaned['Latest Action'].str.contains("referred to the committee", case=False), 1, 0)
print(house_cleaned["bill_referred_committee"].value_counts())
print(f'For the 113th to 118th House passed {(round((4405/34666)*100,2))}% of laws introduced in the House.')

0    34666
1     4405
Name: bill_referred_committee, dtype: int64
For the 113th to 118th House passed 12.71% of laws introduced in the House.


/Users/zackwcrowley/opt/anaconda3/envs/PythonDataOne/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [84]:
house_cleaned.head()

,Bill Type,Congress,Number of Cosponsors,Cosponsor Dems,Cosponsor Reps,Cosponsor Ind,Cosponsor States,Committees,Latest Action,Subject,...,Transportation and Infrastructure,Veterans' Affairs,Ways and Means,Intelligence,Printing,Taxation,Library,Economic,bill_passed,bill_referred_committee
0,H.R.,113,200,200,0,0,46,"House - Judiciary, Energy and Commerce, Educat...",Referred to the Subcommittee on Higher Educati...,Accounting and auditing,...,0,0,0,0,0,0,0,0,0,0
1,H.R.,113,179,179,0,0,42,"House - House Administration, Judiciary, Scien...",Referred to the Subcommittee on Higher Educati...,Administrative law and regulatory procedures,...,0,1,0,0,0,0,0,0,0,0
2,H.R.,113,0,0,0,0,0,NaN,NaN,NaN,...,1,1,1,1,1,1,1,1,1,1
3,H.R.,113,0,0,0,0,0,NaN,NaN,NaN,...,1,1,1,1,1,1,1,1,1,1
4,H.R.,113,200,197,3,0,46,"House - Judiciary, Foreign Affairs, Homeland S...",Motion to Discharge Committee filed by Mr. Gar...,Administrative law and regulatory procedures,...,1,0,1,0,0,0,0,0,0,0


## Save clean house dataset: 

In [85]:
# Save house cleaned dataset:
# house_cleaned.to_csv('../Resources/house_cleaned.csv', index=False)

### Senate Cleaning:

In [86]:
# Committee column recoding to indicator variables:
# Create a list of committees for the Senate:
senate_committees_lst = ["Agriculture, Nutrition, and Forestry", "Appropriations", "Armed Services", "Banking, Housing, and Urban Affairs", "Budget", 
                         "Commerce, Science, and Transportation", "Energy and Natural Resources", "Environment and Public Works", "Finance", 
                         "Foreign Relations", "Health, Education, Labor, and Pensions", "Homeland Security and Governmental Affairs","Judiciary", 
                         "Rules and Administration", "Small Business and Entrepreneurship", "Veterans Affairs", "International Narcotics Control", 
                         "Ethics", "Indian Affairs", "Intelligence", "Printing", "Taxation", "Library", "Economic"]
# Run a for loop to set each committee name to a new column and make a dummy var (case=False makes the str.contains case insensitive)):
for comm in senate_committees_lst:
    senate_cleaned[comm] = np.where(senate_cleaned['Committees'].str.contains(comm, case=False), 1, 0)

senate_cleaned.head()

,Bill Type,Congress,Number of Cosponsors,Cosponsor Dems,Cosponsor Reps,Cosponsor Ind,Cosponsor States,Committees,Latest Action,Subject,...,Small Business and Entrepreneurship,Veterans Affairs,International Narcotics Control,Ethics,Indian Affairs,Intelligence,Printing,Taxation,Library,Economic
0,S.,113,15,15,0,0,12,Senate - Judiciary,Read twice and referred to the Committee on th...,Border security and unlawful immigration,...,0,0,0,0,0,0,0,0,0,0
1,S.,113,16,16,0,0,12,Senate - Judiciary,Read twice and referred to the Committee on th...,Child health,...,0,0,0,0,0,0,0,0,0,0
2,S.,113,16,15,0,1,11,"Senate - Health, Education, Labor, and Pensions",Read twice and referred to the Committee on He...,Child safety and welfare,...,0,0,0,0,0,0,0,0,0,0
3,S.,113,14,14,0,0,12,"Senate - Commerce, Science, and Transportation",Read twice and referred to the Committee on Co...,Academic performance and assessments,...,0,0,0,0,0,0,0,0,0,0
4,S.,113,31,30,0,1,24,Senate - Judiciary,Read twice and referred to the Committee on th...,Crime and Law Enforcement,...,0,0,0,0,0,0,0,0,0,0


In [89]:
# senate_cleaned.head()
# senate_cleaned.dtypes
# senate_cleaned.apply(pd.value_counts)
# senate_cleaned['Committees'].unique()

,Bill Type,Congress,Number of Cosponsors,Cosponsor Dems,Cosponsor Reps,Cosponsor Ind,Cosponsor States,Committees,Latest Action,Subject,...,Veterans Affairs,International Narcotics Control,Ethics,Indian Affairs,Intelligence,Printing,Taxation,Library,Economic,bill_passed
0,S.,113,15,15,0,0,12,Senate - Judiciary,Read twice and referred to the Committee on th...,Border security and unlawful immigration,...,0,0,0,0,0,0,0,0,0,0
1,S.,113,16,16,0,0,12,Senate - Judiciary,Read twice and referred to the Committee on th...,Child health,...,0,0,0,0,0,0,0,0,0,0
2,S.,113,16,15,0,1,11,"Senate - Health, Education, Labor, and Pensions",Read twice and referred to the Committee on He...,Child safety and welfare,...,0,0,0,0,0,0,0,0,0,0
3,S.,113,14,14,0,0,12,"Senate - Commerce, Science, and Transportation",Read twice and referred to the Committee on Co...,Academic performance and assessments,...,0,0,0,0,0,0,0,0,0,0
4,S.,113,31,30,0,1,24,Senate - Judiciary,Read twice and referred to the Committee on th...,Crime and Law Enforcement,...,0,0,0,0,0,0,0,0,0,0


In [90]:
# Latest Action coding:
# Get "bill passed into law"
senate_cleaned["bill_passed"] = np.where(senate_cleaned['Latest Action'].str.contains("Became Public Law", case=False), 1, 0)
print(senate_cleaned["bill_passed"].value_counts())
print(f'For the 113th to 118th Senate passed {(round((455/14325)*100,2))}% of laws introduced in the Senate.')

0    14325
1      455
Name: bill_passed, dtype: int64
For the 113th to 118th Senate passed 3.18% of laws introduced in the Senate.


In [92]:
# Latest Action coding:
# Get "referred to the committee" and name it "bill_referred_committee" set as dummy var:
senate_cleaned["bill_referred_committee"] = np.where(senate_cleaned['Latest Action'].str.contains("referred to the committee", case=False), 1, 0)
print(senate_cleaned["bill_referred_committee"].value_counts())
print(f'For the 113th to 118th Senate passed {(round((3286/11494)*100,2))}% of laws introduced in the Senate.')

1    11494
0     3286
Name: bill_referred_committee, dtype: int64
For the 113th to 118th Senate passed 28.59% of laws introduced in the Senate.


## Save clean house dataset: 

In [95]:
# Save house cleaned dataset:
# senate_cleaned.to_csv('../Resources/senate_cleaned.csv', index=False)